<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Adv_ML_project/blob/main/LM_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this milestone, we will be categorizing whether a given text belongs to one of the Indian languages from a set of {Hindi, Malayalam, Kannada, Tamil, Bengali}. For this we will create a Bag of Vectors for each language. Here are the steps that we aim to implement-

Collate the dataset for each language. We will use the Hindi, Kannada, Tamil, Malayalam, and Bengali Wikipedia articles to train our model. Use publicly available tokenizers (or generate a new one if possible - stretch goal) to create a Bag of Words Create a Bag of Words classifier using linear layers Train the Classifier

For Language Identification, a lot of work has been done to classify and identify languages with latin as the predominant script. Some work has also been done to identify Chinese, Japanese, and Korean languages. Jauhiaihen et al has done a detailed survey on the literature that exists for Language Identification tasks. Kerwin (2006) used character frequencies as feature vectors. In a feature vector, each feature vector f has its own integer value. Raw frequency and relative frequency for each feature is calculated for each language. For our project, we will be using something very similar.

There are several intuitive techniques that are used to classify languages- Position of words- Kumar et al. (2015) used the position of the current word in word-level LI. Dictionary of unique words: Unique word dictionaries include only those words of the language, that do not belong to the other languages targeted by the language identifier. Discriminating words Kolkus (2009) used the most relevant words for each language

In [59]:
!pip install indic-nlp-library
!pip install torch
!pip install torchtext


In [15]:
import sys
import os
from google.colab import drive 
drive.mount('/content/gdrive')
%cd /content/drive/MyDrive/AML_Project/
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/AML_Project"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 107] Transport endpoint is not connected: '/content/drive/MyDrive/AML_Project/'
/content/drive/MyDrive/Adv_ML_Project


In [22]:

import torch
import torchtext
import spacy
from torchtext import data, datasets
from torchtext.vocab import Vectors
from torch.nn import init
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

Citation for datasets: @inproceedings{kakwani2020indicnlpsuite,
    title={{IndicNLPSuite: Monolingual Corpora, Evaluation Benchmarks and Pre-trained Multilingual Language Models for Indian Languages}},
    author={Divyanshu Kakwani and Anoop Kunchukuttan and Satish Golla and Gokul N.C. and Avik Bhattacharyya and Mitesh M. Khapra and Pratyush Kumar},
    year={2020},
    booktitle={Findings of EMNLP},
}


@article{kunchukuttan2020indicnlpcorpus,
    title={AI4Bharat-IndicNLP Corpus: Monolingual Corpora and Word Embeddings for Indic Languages},
    author={Anoop Kunchukuttan and Divyanshu Kakwani and Satish Golla and Gokul N.C. and Avik Bhattacharyya and Mitesh M. Khapra and Pratyush Kumar},
    year={2020},
    journal={arXiv preprint},


Milestone 1: Develop text classifier of our own

Reference: https://nbviewer.org/url/anoopkunchukuttan.github.io/indic_nlp_library/doc/indic_nlp_examples.ipynb

In [40]:
from indicnlp.tokenize import sentence_tokenize

indic_string="""तो क्या विश्व कप 2019 में मैच का बॉस टॉस है? यानी मैच में हार-जीत में \
टॉस की भूमिका अहम है? आप ऐसा सोच सकते हैं। विश्वकप के अपने-अपने पहले मैच में बुरी तरह हारने वाली एशिया की दो टीमों \
पाकिस्तान और श्रीलंका के कप्तान ने हालांकि अपने हार के पीछे टॉस की दलील तो नहीं दी, लेकिन यह जरूर कहा था कि वह एक अहम टॉस हार गए थे।"""
sentences=sentence_tokenize.sentence_split(indic_string, lang='hi')
for t in sentences:
    print(t)

तो क्या विश्व कप 2019 में मैच का बॉस टॉस है?
यानी मैच में हार-जीत में टॉस की भूमिका अहम है?
आप ऐसा सोच सकते हैं।
विश्वकप के अपने-अपने पहले मैच में बुरी तरह हारने वाली एशिया की दो टीमों पाकिस्तान और श्रीलंका के कप्तान ने हालांकि अपने हार के पीछे टॉस की दलील तो नहीं दी, लेकिन यह जरूर कहा था कि वह एक अहम टॉस हार गए थे।


In [41]:
from indicnlp.tokenize import indic_tokenize  

indic_string='सुनो, कुछ आवाज़ आ रही है। फोन?'

print('Input String: {}'.format(indic_string))
print('Tokens: ')
for t in indic_tokenize.trivial_tokenize(indic_string): 
    print(t)

Input String: सुनो, कुछ आवाज़ आ रही है। फोन?
Tokens: 
सुनो
,
कुछ
आवाज़
आ
रही
है
।
फोन
?


Milestone 2: See text classificaiton using INLTK 

In [ ]:
!kaggle datasets list -s 'hindi-wikipedia'

ref                                                        title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
disisbig/hindi-wikipedia-articles-172k                     Hindi Wikipedia Articles - 172k                    208MB  2019-12-24 05:01:30            393         19  0.6875           
zarajamshaid/language-identification-datasst               Language Identification dataset                      6MB  2018-12-19 18:12:23           2117         36  1.0              
disisbig/hindi-wikipedia-articles-55k                      Hindi Wikipedia Articles - 55k                      63MB  2019-12-24 04:31:32            200          8  0.625            
shivavashishtha/shark-tank-india-dataset                   Shark Tank India Dataset       

In [ ]:
!kaggle datasets download -d 'disisbig/hindi-wikipedia-articles-55k'

 90% 57.0M/63.3M [00:02<00:00, 28.3MB/s]
100% 63.3M/63.3M [00:02<00:00, 31.5MB/s]


The hindi dataset is now sitting in the folder in zip format at '/content/drive/MyDrive/Adv_ML_Project/hindi-wikipedia-articles-55k.zip'

In [ ]:
from zipfile import ZipFile

with ZipFile('hindi-wikipedia-articles-55k.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
# This above code is shit slow. It takes 20 hours to extract. 





In [ ]:
!unzip 'hindi-wikipedia-articles-55k.zip' # This is relatively fast

In [ ]:
!kaggle datasets list -s 'bengali-wikipedia'

ref                                            title                                   size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------  -------------------------------------  -----  -------------------  -------------  ---------  ---------------  
disisbig/bengali-wikipedia-articles            Bengali Wikipedia Articles             105MB  2019-12-25 04:35:21            135          6  0.5              
nrkapri/rabindranath-tagore-online-variorum    Rabindranath Tagore Online Variorum    173MB  2020-06-07 20:04:16             50          9  1.0              
nbroad/muril-large-pt                          MuRIL Large pt                           2GB  2021-10-16 14:35:24             61         12  0.5              
aagalib/complete-works-of-rabindranath-tagore  Complete Works of Rabindranath Tagore   43MB  2022-01-04 16:05:23             28          8  0.9411765        
zzy990106/murilbasecased                       muril

In [45]:
!kaggle datasets download -d 'disisbig/bengali-wikipedia-articles'

bengali-wikipedia-articles.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip 'bengali-wikipedia-articles.zip' # This is relatively fast

In [ ]:
!kaggle datasets list -s 'kannada-wikipedia'

ref                                            title                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------  -----------------------------  -----  -------------------  -------------  ---------  ---------------  
disisbig/kannada-wikipedia-articles            Kannada Wikipedia Articles      81MB  2019-12-25 04:38:53            168          1  0.5              
nbroad/muril-large-pt                          MuRIL Large pt                   2GB  2021-10-16 14:35:24             61         12  0.5              
zzy990106/murilbasecased                       muril-base-cased                 2GB  2021-08-21 16:36:07             26          8  0.6875           
sizlingdhairya1/iiit-spoken-language-datasets  IIIT Spoken Language Datasets  920MB  2022-05-02 08:37:28              2          0  0.3125           
nbroad/muril-large-tf                          MuRIL Large tf                   2GB  2021-10-16 14:2

In [ ]:
!kaggle datasets list -s 'malayalam-wikipedia'

ref                                            title                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------  -----------------------------  -----  -------------------  -------------  ---------  ---------------  
disisbig/malayalam-wikipedia-articles          Malayalam Wikipedia Articles    19MB  2019-12-25 04:39:06            121          6  0.5              
nbroad/muril-large-pt                          MuRIL Large pt                   2GB  2021-10-16 14:35:24             61         12  0.5              
zzy990106/murilbasecased                       muril-base-cased                 2GB  2021-08-21 16:36:07             26          8  0.6875           
sizlingdhairya1/iiit-spoken-language-datasets  IIIT Spoken Language Datasets  920MB  2022-05-02 08:37:28              2          0  0.3125           
nbroad/muril-large-tf                          MuRIL Large tf                   2GB  2021-10-16 14:2

In [ ]:
!kaggle datasets list -s 'tamil-wikipedia'

ref                                                title                                       size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------  -----------------------------------------  -----  -------------------  -------------  ---------  ---------------  
disisbig/tamil-wikipedia-articles                  Tamil Wikipedia Articles                    95MB  2019-12-25 04:30:22            172         12  0.64705884       
jrobischon/wikipedia-movie-plots                   Wikipedia Movie Plots                       30MB  2018-10-15 19:59:54          14585        432  0.88235295       
sudalairajkumar/tamil-nlp                          Tamil NLP                                    3MB  2019-03-11 06:29:11           1284        100  1.0              
zarajamshaid/language-identification-datasst       Language Identification dataset              6MB  2018-12-19 18:12:23           2117         36  1.0              
prav

In [20]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
train_kn = pd.read_csv('/content/gdrive/MyDrive/AML_Project/kn/kn-train.csv')
test_kn = pd.read_csv('/content/gdrive/MyDrive/AML_Project/kn/kn-test.csv')
valid_kn = pd.read_csv('/content/gdrive/MyDrive/AML_Project/kn/kn-valid.csv')

In [46]:
test_kn

,sports,"ವಿಂಡೀಸ್‌ ತಂಡಕ್ಕೆ ಮರಳಿದ ಸುನೀಲ್‌, ಪೊಲ್ಲಾರ್ಡ್‌\n‌ಭಾರತ ವಿರುದ್ಧ ಟಿ–20 ಸರಣಿಗೆ ತಂಡ ಪ್ರಕಟ\nವಿಂಡೀಸ್‌ ತಂಡಕ್ಕೆ ಮರಳಿದ ಸುನೀಲ್‌, ಪೊಲ್ಲಾರ್ಡ್‌\nಪ್ರಜಾವಾಣಿ ವಾರ್ತೆ\nPublished:\n23 ಜುಲೈ 2019, 17:00 IST\nUpdated:\n23 ಜುಲೈ 2019, 17:00 IST\nಅಕ್ಷರ ಗಾತ್ರ :\nಆ\nಆ\nಸೇಂಟ್‌ ಜಾನ್ಸ್‌, ಆ್ಯಂಟೀಗಾ (ಪಿಟಿಐ): ಭಾರತ ವಿರುದ್ಧ ಮೂರು ಪಂದ್ಯಗಳ ಟಿ–20 ಸರಣಿಯಲ್ಲಿ ಆಡುವ ಮೊದಲ ಎರಡು ಪಂದ್ಯಗಳಿಗೆ ವೆಸ್ಟ್‌ ಇಂಡೀಸ್‌ ತಂಡವನ್ನು ಪ್ರಕಟಿಸಲಾಗಿದೆ. ಹಿರಿಯ ಆಟಗಾರರಾದ ಸುನೀಲ್‌ ನಾರಾಯಣ್‌ ಮತ್ತು ಕೀರನ್‌ ಪೊಲ್ಲಾರ್ಡ್‌ ಅವರನ್ನು ಮರಳಿ ಕರೆಸಿಕೊಳ್ಳಲಾಗಿದೆ.\nಕಾರ್ಲೋಸ್‌ ಬ್ರಾತ್‌ವೇಟ್‌ ನಾಯಕತ್ವದ 14 ಮಂದಿಯ ತಂಡವನ್ನು ಮಂಗಳವಾರ ಪ್ರಕಟಿಸಲಾಯಿತು. ವಿಕೆಟ್‌ ಕೀಪರ್‌– ಬ್ಯಾಟ್ಸ್‌ಮನ್‌ ಅಂಥೋನಿ ಬ್ರಾಂಬಲ್ ತಂಡದಲ್ಲಿರುವ ಏಕೈಕ ಹೊಸಮುಖ. 28 ವರ್ಷದ ಬ್ರಾಂಬಲ್‌, ನಿಕೋಲಸ್‌ ಪೂರನ್‌ಗೆ ಬ್ಯಾಕ್‌ಅಪ್‌ ವಿಕೆಟ್‌ ಕೀಪರ್ ಆಗಿದ್ದಾರೆ.\nಮೊದಲ ಎರಡು ಪಂದ್ಯಗಳು ಆಗಸ್ಟ್‌ 3 ಮತ್ತು 4ರಂದು ಅಮೆರಿಕದ ಫ್ಲಾರಿಡಾದ ಲಾಡರ್‌ಹಿಲ್‌ನ ಬ್ರೊವಾರ್ಡ್‌ ಕೌಂಟಿ ಸ್ಟೇಡಿಯಂನಲ್ಲಿ ನಡೆಯಲಿವೆ. ಗಯಾನಾದ ನ್ಯಾಷನಲ್‌ ಸ್ಟೇಡಿಯಂನಲ್ಲಿ ಆಗಸ್ಟ್‌ 6ರಂದು ಅಂತಿಮ ಪಂದ್ಯ ನಡೆಯಲಿದೆ ಎಂದು ವೆಸ್ಟ್‌ ಇಂಡೀಸ್‌ ಕ್ರಿಕೆಟ್‌ ಮಂಡಳಿ ತಿಳಿಸಿದೆ. ಈ ಸರಣಿಯ ಬಳಿಕ ಮೂರು ಏಕದಿನ ಮತ್ತು ಎರಡು ಟೆಸ್ಟ್‌ ಪಂದ್ಯಗಳ ಸರಣಿ ಆಡಬೇಕಿದೆ.\nಆಲ್‌ರೌಂಡರ್‌ ಆ್ಯಂಡ್ರೆ ರಸೆಲ್‌ ತಂಡದಲ್ಲಿ ಅವಕಾಶ ಪಡೆದರೂ, ಅವರು ದೈಹಿಕ ಕ್ಷಮತೆ ಪರೀಕ್ಷೆ ತೇರ್ಗಡೆ ಆಗಬೇಕಾಗಿದೆ. ಇತ್ತೀಚಿನ ವಿಶ್ವಕಪ್‌ ಸಂದರ್ಭದಲ್ಲಿ ರಸೆಲ್‌, ಎಡ ಮಂಡಿ ನೋವು ಕಾಡುತ್ತಿದ್ದ ಪರಿಣಾಮ ಶಸ್ತ್ರಚಿಕಿತ್ಸೆ ಮಾಡಿಸಿಕೊಳ್ಳಲು ಅರ್ಧದಲ್ಲೇ ಸ್ವದೇಶಕ್ಕೆ ಮರಳಿದ್ದರು.\nಕೀರನ್‌ ಪೊಲ್ಲಾರ್ಡ್‌\nಅನುಭವಿ ಓಪನರ್‌ ಕ್ರಿಸ್‌ ಗೇಲ್‌, ಕೆನಡಾದಲ್ಲಿ ಜಿಟಿ–20 ಟೂರ್ನಿಯಲ್ಲಿ ಆಡಬೇಕಾಗಿರುವ ಕಾರಣ ಈ ಸರಣಿಗೆ ಲಭ್ಯವಿರುವುದಿಲ್ಲ ಎಂದು ಆಯ್ಕೆ ಸಮಿತಿಗೆ ತಿಳಿಸಿದ್ದಾಗಿ ಸಮಿತಿಯ ಹಂಗಾಮಿ ಅಧ್ಯಕ್ಷ ರಾಬರ್ಟ್‌ ಹೇನ್ಸ್‌ ತಿಳಿಸಿದರು.\nಮುಂದಿನ ವರ್ಷ ಆಸ್ಟ್ರೇಲಿಯಾದಲ್ಲಿ ನಡೆಯಲಿರುವ ಟಿ–20 ವಿಶ್ವಕಪ್‌ಗೆ ಸಿದ್ಧತೆಯ ಉದ್ದೇಶ ದೃಷ್ಟಿಯಲ್ಲಿಟ್ಟುಕೊಂಡು ಎಡಗೈ ಆರಂಭ ಆಟಗಾರ ಜಾನ್‌ ಕ್ಯಾಂಪ್‌ಬೆಲ್‌ ಮತ್ತು ಎಡಗೈ ಸ್ಪಿನ್ನರ್‌ ಖಾರಿ ಪಿಯರೆ ಅವರನ್ನೂ ಸೇರ್ಪಡೆ ಮಾಡಲಾಗಿದೆ.\n‘ತಂಡ ಅನುಭವಿ ಮತ್ತು ಯುವಮುಖಗಳೊಂದಿಗೆ ಸಂತುಲಿತವಾಗಿದೆ. ಇದು ಭಾರತ ವಿರುದ್ಧ ಸರಣಿ ಉದ್ದೇಶದಿಂದ ಮಾತ್ರ ಆಯ್ಕೆ ಮಾಡಿದ ತಂಡವಲ್ಲ. ಮುಂದಿನ ಟಿ–20 ವಿಶ್ವಕಪ್‌ಗೆ ತಂಡ ಸಂಯೋಜನೆ ಮಾಡುವ ಉದ್ದೇಶವೂ ಇದರಲ್ಲಿದೆ’ ಎಂದು ಅವರು ಹೇಳಿದರು.\n‘ತಂಡಕ್ಕೆ ಆಯ್ಕೆಯಾಗಿರುವ ಹೆಚ್ಚಿನ ಆಟಗಾರರು ಐಪಿಎಲ್‌ ಆಡಿದವರು. ಅವರು ಭಾರತದ ಆಟಗಾರರನ್ನು ಚೆನ್ನಾಗಿ ಬಲ್ಲರು. ಸ್ವದೇಶದಲ್ಲೇ ಆಡುವುದರಿಂದ ನಮಗೆ ಸ್ವಲ್ಪ ಅನುಕೂಲವಿದೆ. ಆದರೆ ಅಂತಿಮವಾಗಿ ತಂಡದ ಆಟಗಾರರು ಗೆಲ್ಲಿಸುವುದು ಮುಖ್ಯವಾಗುತ್ತದೆ’ ಎಂದರು.\nಆಫ್‌ ಬ್ರೇಕ್‌ ಬೌಲರ್ ಸುನೀಲ್‌ ನಾರಾಯಣ್‌, ಎರಡು ವರ್ಷದ ಹಿಂದೆ ಇಂಗ್ಲೆಂಡ್‌ ಪ್ರವಾಸದ ವೇಳೆ ಕೊನೆಯ ಬಾರಿ ಟಿ–20 ಅಂತರರಾಷ್ಟ್ರೀಯ ಪಂದ್ಯ ಆಡಿದ್ದರು. ಪೊಲ್ಲಾರ್ಡ್‌, ಕಳೆದ ನವೆಂಬರ್‌ನಲ್ಲಿ ಭಾರತ ಪ್ರವಾಸದ ವೇಳೆ ಕೊನೆಯ ಬಾರಿ ಟಿ–20 ಪಂದ್ಯ ಆಡಿದ್ದರು. \nತಂಡ ಇಂತಿದೆ: ಕಾರ್ಲೋಸ್‌ ಬ್ರಾತ್‌ವೇಟ್‌ (ಕ್ಯಾಪ್ಟನ್‌), ಸುನೀಲ್‌ ನಾರಾಯಣ್‌, ಕೀಮೊ ಪಾಲ್‌, ಖಾರಿ ಪಿಯರೆ, ಕೀರನ್‌ ಪೊಲ್ಲಾರ್ಡ್‌, ನಿಕೋಲಸ್‌ ಪೂರನ್‌ (ವಿಕೆಟ್‌ ಕೀಪರ್‌), ರೊವ್‌ಮನ್‌ ಪೊವೆಲ್‌, ಆ್ಯಂಡ್ರೆ ರಸೆಲ್‌, ಒಷಾನೆ ಥಾಮಸ್‌, ಆಂಥೋನಿ ಬ್ರಾಂಬಲ್‌ (ವಿಕೆಟ್‌ ಕೀಪರ್‌), ಜಾನ್‌ ಕ್ಯಾಂಪ್‌ಬೆಲ್‌, ಶೆಲ್ಟನ್‌ ಕಾಟ್ರೆಲ್‌, ಶಿಮ್ರಾನ್‌ ಹೆಟ್ಮೆಯರ್‌, ಎವಿನ್‌ ಲೂಯಿಸ್‌.\n0 Post Comments (+)"
0,entertainment,ರಂಗದಲ್ಲಿ ಕಮಲಾದೇವಿ ಚಟ್ಟೋಪಾಧ್ಯಾಯ\nkamaladevi cha...
1,entertainment,"Bengaluru, First Published 23, Nov 2018, 9:38 ..."
2,sports,"ಅಧ್ಯಯನ, ಅಭ್ಯಾಸದ ಗೊಂದಲದಲ್ಲಿ ಹಿಮಾ ದಾಸ್‌\nಅಧ್ಯಯನ,..."
3,lifestyle,ದೇಶ ಜೀವನದ ಕೊನೆಯ ಹಂತದಲ್ಲಿ ಆರೋಗ್ಯದ ದುಃಸ್ಥಿತಿ!\nJ...
4,entertainment,"Bangalore, First Published 11, Jun 2019, 10:40..."
...,...,...
2994,entertainment,ವೀಕೆಂಡ್‌ ವಿಥ್‌ ರಮೇಶ್‌; ನಾಲ್ಕನೇ ಸೀಸನ್‌ ಸದ್ಯದಲ್ಲ...
2995,entertainment,"Bangalore, First Published 15, Jul 2019, 8:58 ..."
2996,entertainment,ಅಬ್ಬಾ...! ಕರೀನಾ ಕಪೂರ್ ಅಭಿಮಾನಿ ಜೈಲು ಸೇರಿದ್ಯಾಕೆ...
2997,lifestyle,ದೇಹದ ರಕ್ತ ಹೆಪ್ಪುಗಟ್ಟುವಿಕೆಯ ಬಗ್ಗೆ ಇರುವ ಕುತೂಹಲಕ...


In [24]:
!pip install spacy


In [30]:
spacy_kn = torchtext.data.utils.get_tokenizer('spacy', language='kn')

OSError: ignored

In [56]:
#Can we use spacy to tokeknize ourselves? Yes
from spacy.lang.kn import Kannada
nlp_kn = Kannada()
print(nlp_kn.lang, [token.is_stop for token in nlp_kn("ಇಂಡೀಸ್‌ ಹಲವು")])
doc = nlp_kn("ವಿರುದ್ಧ ಮೂರು ಪಂದ್ಯಗಳ ಟಿ–20 ಸರಣಿಯಲ್ಲಿ ಆಡುವ ಮೊದಲ ಎರಡು ಪಂದ್ಯಗಳಿಗೆ ವೆಸ್ಟ್‌ ")
for token in doc:
  print(token)

kn [False, True]
ವಿರುದ್ಧ
ಮೂರು
ಪಂದ್ಯಗಳ
ಟಿ–20
ಸರಣಿಯಲ್ಲಿ
ಆಡುವ
ಮೊದಲ
ಎರಡು
ಪಂದ್ಯಗಳಿಗೆ
ವೆಸ್ಟ್‌


In [62]:
#Let's try tokenizer designed by Google and IIT Madras. Yes
from indicnlp.tokenize import indic_tokenize 
kannada_string = "ವಿರುದ್ಧ ಮೂರು ಪಂದ್ಯಗಳ ಟಿ–20 ಸರಣಿಯಲ್ಲಿ ಆಡುವ ಮೊದಲ ಎರಡು ಪಂದ್ಯಗಳಿಗೆ ವೆಸ್ಟ್‌ "
for t in indic_tokenize.trivial_tokenize(kannada_string): 
    print(t)

ವಿರುದ್ಧ
ಮೂರು
ಪಂದ್ಯಗಳ
ಟಿ–20
ಸರಣಿಯಲ್ಲಿ
ಆಡುವ
ಮೊದಲ
ಎರಡು
ಪಂದ್ಯಗಳಿಗೆ
ವೆಸ್ಟ್‌


In [57]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab

Milestone 3: Use transfer learning on INLTK for classification accuracy improvement

Credit for datasets: @inproceedings{kakwani2020indicnlpsuite,
    title={{IndicNLPSuite: Monolingual Corpora, Evaluation Benchmarks and Pre-trained Multilingual Language Models for Indian Languages}},
    author={Divyanshu Kakwani and Anoop Kunchukuttan and Satish Golla and Gokul N.C. and Avik Bhattacharyya and Mitesh M. Khapra and Pratyush Kumar},
    year={2020},
    booktitle={Findings of EMNLP},
}

